# Eat Safe, Love

## Notebook Set Up

In [26]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [12]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [13]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [14]:
# review the collections in our database
print(db.list_collection_names())


['establishments']


In [15]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [16]:
# Find the establishments with a hygiene score of 20

h_score_query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(h_score_query))


41


In [27]:
# Define the hygiene score query
h_score_query = {"scores.Hygiene": 20}

# Define the additional group stage
group_query = {
    "$group": {
        "_id": "$_id",  # You can use any grouping key you want
        "count": {"$sum": 1},
        "first_doc": {"$first": "$$ROOT"}  # Include the entire document in the result
    }
}

# Combine the queries into a list
pipeline = [
    {"$match": h_score_query},  # Match documents with the specified hygiene score
    group_query  # Additional group stage
]

# Execute the aggregation pipeline
results = list(establishments.aggregate(pipeline))

# Extract the original documents from the result
original_documents = [result["first_doc"] for result in results]

# Convert the result to a Pandas DataFrame

establishments_df = pd.DataFrame(original_documents)

# Display the number of rows in the DataFrame
print(len(establishments_df))

# Display the first 10 rows of the DataFrame
print(establishments_df.head(10))



41
                        _id   FHRSID  ChangesByServerID  \
0  65befda47699be012c29adb3  1411729                  0   
1  65befda57699be012c29c0ad  1498084                  0   
2  65befda57699be012c29c19c  1063454                  0   
3  65befda57699be012c29c213  1130121                  0   
4  65befda47699be012c29b00d  1060293                  0   
5  65befda57699be012c29c517  1530599                  0   
6  65befda57699be012c29be98  1107748                  0   
7  65befda57699be012c29c4b8   640816                  0   
8  65befda47699be012c29a476  1384371                  0   
9  65befda27699be012c296cf5  1537089                  0   

  LocalAuthorityBusinessID                 BusinessName  \
0           21/00562/MIXED                       Londis   
1            22/00797/FOOD                   Sisko Cafe   
2            18/00285/FOOD       Magazin Romanesc Diana   
3            19/00249/FOOD                Bali Maamalas   
4             PI/000206588                   Costcut

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [58]:
# Find the establishments with London as the Local Authority and RatingValue greater than or equal to 4
london_boroughs = establishments.distinct("LocalAuthorityName", {"LocalAuthorityName": {"$regex": "London", "$options": "i"}})

query = {
    "$and": [
        {"LocalAuthorityName": {"$in": london_boroughs}},
        {"RatingValue": {"$gte": 4}}
    ]
}

# Find establishments matching the query
matching_establishments = establishments.find(query)

# Use count_documents to display the number of matching documents
count = establishments.count_documents(query)
print(f"Number of documents in London with RatingValue >= 4: {count}")

# Display the first few matching documents using pprint
print("Matching documents:")
for doc in matching_establishments:
    pprint(doc)

Number of documents in London with RatingValue >= 4: 33
Matching documents:
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65befda27699be012c2972c9'),
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'https:/

In [65]:
# Find establishments matching the query
matching_establishments = establishments.find(query)

# Convert the result to a Pandas DataFrame
matching_establishments_df = pd.DataFrame(list(matching_establishments))


# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(matching_establishments_df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(matching_establishments_df.head(10))


Number of rows in the DataFrame: 33
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  65befda27699be012c2972c9   621707                  0   
1  65befda37699be012c2975ed  1130836                  0   
2  65befda37699be012c298135   293783                  0   
3  65befda37699be012c298f39   294474                  0   
4  65befda37699be012c298f3c  1315095                  0   
5  65befda47699be012c299641   294900                  0   
6  65befda47699be012c29b74f   293756                  0   
7  65befda47699be012c29bb15   878523                  0   
8  65befda47699be012c29bb33   293772                  0   
9  65befda47699be012c29bb34   295107                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000025307                             Charlie's   
1             PI/000034075               Mv City Cruises Erasmus   
2             PI/000002614             Benfleet Motor Yacht Club   
3         

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [35]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
# Find the geocode (latitude and longitude) for the Penang Flavours restaurant
penang_flavours_geocode = establishments.find_one({"BusinessName": "Penang Flavours"})['geocode']

degree_search = 0.01
latitude = penang_flavours_geocode['latitude']
longitude = penang_flavours_geocode['longitude']

query = {
    "$and": [
        {"RatingValue": 5},
        {
            "geocode.latitude": {
                "$gte": latitude - 0.01,
                "$lte": latitude + 0.01
            }
        },
        {
            "geocode.longitude": {
                "$gte": longitude - 0.01,
                "$lte": longitude + 0.01
            }
        }
    ]
}
sort = [("scores.Hygiene", 1)]  # 1 for ascending order (lowest hygiene score first)


# Print the results
top5_near_penang = establishments.find(query).sort(sort).limit(5)
for doc in top5_near_penang:
    pprint(doc)


{'AddressLine1': '130 - 132 Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'Volunteer',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4646.965634598608,
 'FHRSID': 694609,
 'LocalAuthorityBusinessID': 'PI/000116619',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1JQ',
 'RatingDate': '2019-08-05T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65befda47699be012c29b108'),
 'geocode': {'latitude': 51.4873437, 'longitude': 0.09208},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/694609',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate'

In [51]:
# Convert result to Pandas DataFrame
near_penang_df = pd.DataFrame( establishments.find(query).sort(sort))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(near_penang_df)}")
print(near_penang_df.head(5))

Number of rows in the DataFrame: 33
                        _id   FHRSID  ChangesByServerID  \
0  65befda37699be012c2975ed  1130836                  0   
1  65befda37699be012c298135   293783                  0   
2  65befda37699be012c298f39   294474                  0   
3  65befda37699be012c298f3c  1315095                  0   
4  65befda47699be012c299641   294900                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000034075               Mv City Cruises Erasmus   
1             PI/000002614             Benfleet Motor Yacht Club   
2             PI/000014647              Tilbury Seafarers Centre   
3             PI/000036464  Coombs Catering t/a The Lock and Key   
4             PI/000019066                            Mv Valulla   

              BusinessType  BusinessTypeID  \
0  Other catering premises            7841   
1  Other catering premises            7841   
2  Restaurant/Cafe/Canteen               1   
3  Restaurant/Ca

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [30]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

hygiene_score_0 = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print(f"Total number of establishments with a hygiene score of 0: {sum(doc['count'] for doc in hygiene_score_0)}")

# Print the first 10 results
pprint(hygiene_score_0[:10])


Total number of establishments with a hygiene score of 0: 16827
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [31]:
# Convert the result to a Pandas DataFrame
hygiene_score_0_df = pd.DataFrame(hygiene_score_0)

# Display the number of rows in the DataFrame
print(len(hygiene_score_0_df))

# Display the first 10 rows of the DataFrame
print(hygiene_score_0_df.head(10))


55
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
